In [1]:
from typing import Tuple
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader

import aim

In [11]:
import os, sys
sys.path.append(os.path.abspath('..'))

from src.models.unet import UNet
from src.training.metrics import evaluate, dice_loss, dice
from src.training.train import train
from src.data.datasets import ACDCDataset

In [4]:
dataset = ACDCDataset(path='../../training/', verbose=1)

Loaded saved dataset from /worskpace/tagroi/checkpoints/acdc_dataset.pt


In [9]:
train_set, val_set, _ = random_split(dataset, [8, 4, 940], generator=torch.Generator().manual_seed(42))
loader_train = DataLoader(train_set, batch_size=4, shuffle=True)
loader_val = DataLoader(val_set, batch_size=4, shuffle=False)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = UNet(n_channels=1, n_classes=4, bilinear=True).double()

if device.type == 'cuda':
    model = nn.DataParallel(model).to(device)
    model.n_classes = model.module.n_classes

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [15]:
dice_score = torch.zeros(4)

In [19]:
for epoch in range(40):
    
    acc_loss = 0.

    model.train()

    for inputs, targets in loader_train:
        # move to device
        # target is index of classes
        inputs, targets = inputs.to(device), targets.long().to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, targets) + dice_loss(outputs, targets, True)
        
        dice_score += dice(outputs, targets)
        acc_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f'======= Epoch {epoch}')
    print(f'Accumulated loss {acc_loss}')

    train_perf = dice_score / len(loader_train)
    avg_dice = train_perf.mean()
    print(f'Training performance {train_perf}, {avg_dice}')

    val_perf = evaluate(model, loader_val, device)
    avg_val_dice = val_perf.mean()
    print(f'Validation performance {val_perf}, {avg_val_dice}')

======= Epoch 0
Accumulated loss 0.8283676990545923
Training performance tensor([44.5099, 12.0919,  4.7984, 11.5418]), 18.235490798950195


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 1
Accumulated loss 1.032632250051536
Training performance tensor([45.5054, 12.3852,  5.3171, 12.1109]), 18.829662322998047


Validation performance tensor([0.9568, 0.0658, 0.0000, 0.0000]), 0.2556353807449341
======= Epoch 2
Accumulated loss 1.0901274357167152
Training performance tensor([46.5008, 12.6119,  5.8070, 12.6817]), 19.400360107421875


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 3
Accumulated loss 1.087571846049049
Training performance tensor([47.4928, 12.8928,  6.1683, 13.3587]), 19.97814178466797


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 4
Accumulated loss 1.0583362358629944
Training performance tensor([48.4848, 13.2248,  6.6707, 13.9041]), 20.57109260559082


...

Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 36
Accumulated loss 0.2859986650074239
Training performance tensor([80.3173, 35.2179, 26.1549, 36.8359]), 44.631507873535156


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 37
Accumulated loss 0.24214123561873727
Training performance tensor([81.3151, 36.1400, 26.9825, 37.7624]), 45.55000305175781


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 38
Accumulated loss 0.23200964322905798
Training performance tensor([82.3129, 37.0566, 27.8264, 38.6968]), 46.47315216064453


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
======= Epoch 39
Accumulated loss 0.22857779994756433
Training performance tensor([83.3106, 37.9857, 28.6558, 39.6358]), 47.396968841552734


Validation performance tensor([0.9851, 0.0000, 0.0000, 0.0000]), 0.24627509713172913
